In [43]:
from math import exp
from pprint import pprint
from random import shuffle

from tqdm import tqdm
import time
import os

import random
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import openai

os.getcwd()

'c:\\Users\\G34371231\\OneDrive - The George Washington University\\Desktop\\self_recognition'

In [44]:
# Set the seed for reproducibility
random.seed(123)
all_words = [] #need to have different lists depending on method
all_syn = []

In [45]:
def ind_syn_ChatGPT(replacement_word):
    #just returns the synonym provided by ChatGPT 
    #may wish to try system message where i describe desire behavior rather than in the prompt.
    
    openai.api_key = os.getenv("OPENAI_API_KEY")

    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f'Provide a synonym for the word "{replacement_word}". Your response should strictly be the synonym, so that it may immediately replace the original word within some sentence. If you don not find a synonym just return the original word.'
            }
        ]
)

    #synonym = completion['choices'][0]['message']['content'].strip()
    #synonym = completion.choices[0].message['content'].strip()
    synonym = completion.choices[0].message.content.lower()
    return synonym

In [46]:
def syn_from_contxt_ChatGPT(replacement_phrase):
    #performs synonym replacement with context. The replacement_phrase can be the entire sentence of the word to be replaced, or it can be a smaller phrase.
    
    openai.api_key = os.getenv("OPENAI_API_KEY") # nosec
        
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that rewrites phrases by replacing words surrounded by square brackets with synonyms while preserving context and meaning."},
            {
                "role": "user",
                "content": f' "There are word(s) in this phrase surrounded by square brackets []. Replace the words with their synonyms and get rid of the brackets. Your response is strictly the new phrase containing the synonyms. The phrase is: {replacement_phrase}'
            }
        ]
)
    synonym = completion.choices[0].message.content
    return synonym

In [47]:
def get_synonyms(word):
    """
    Finds and returns synonyms for a given word using WordNet.
    This function takes a word as input, searches for its synonyms 
    using the WordNet synsets, and returns a list of synonyms.

    Args:
        word (str): The word for which to find synonyms.
    Returns:
        list: A list of synonyms for the input word. Returns an 
              empty list if no synonyms are found.
    """
    synonyms = []
    for syn in wordnet.synsets(word):
        for name in syn.lemma_names():
            # Exclude the original word to avoid replacing it with itself
            name = name.replace('_',' ')
            if name.lower() != word.lower():
                synonyms.append(name)
    return synonyms

In [48]:
def sample_words(words_alpha, num_words_to_replace):
    # Randomly sample words to replace - i use 2x words just to account for words without synonym
    idx_words = random.sample(list(enumerate(words_alpha)), min(2*num_words_to_replace, len(words_alpha)))
    chosen_indices = []
    words_to_replace = []
    for pair in idx_words:
        chosen_indices.append(pair[0])
        words_to_replace.append(pair[1])  
      
    #idx_words is the list of words selected paired with their index in original sentence/phrase
    #example: "This is a test sentence to see which words get sampled, and what is returned by the replacement function."
    #(1, 'is'), (8, 'words'), (2, 'a'), (13, 'is'), (4, 'sentence'), (18, 'function'), (0, 'This'), (6, 'see'), (17, 'replacement'), (10, 'sampled')
    return chosen_indices, words_to_replace
   

In [49]:
def replace_words_WordNet(sentence, num_words_to_replace):
    """
    Replaces a specified number of words in a sentence with their synonyms using
    the WordNet library, and then using ChatGPT. 

    This function takes a sentence and an integer specifying the number of words 
    to replace with synonyms. It randomly samples 2x the required number of words 
    to ensure replacements are possible even if some words do not have synonyms.
    It uses the `get_synonyms` function to find synonyms for each sampled word,
    and replaces words in the sentence until the specified number is reached.

    Args:
        sentence (str): The input sentence from which words will be replaced.
        num_words_to_replace (int): The number of words in the sentence to be replaced by synonyms.

    Returns:
        str: The modified sentence with the specified number of words replaced by synonyms.
    """
    
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Filter out non-alphabetic tokens (like punctuation)
    words_alpha = [word for word in words if word.isalpha()]
    
    # Randomly sample words to replace - i use 2x words just to account for words without synonym
    indices, words_to_replace = sample_words(words_alpha, num_words_to_replace)
    print(words_to_replace)
    print(indices)
    # Create a new sentence with synonyms replaced
    words_replaced = 0
    new_sentence = []
    for word in words:
        if word in words_to_replace and words_alpha.index(word) in indices:
            synonyms = get_synonyms(word)
            if synonyms and words_replaced < num_words_to_replace:
                # Replace with a random synonym
                new_word = random.choice(synonyms)
                new_sentence.append(new_word)
                #operational
                all_words.append(word)
                all_syn.append(synonyms)
                words_replaced +=1
            else:
                # If no synonym is found, keep the original word
                new_sentence.append(word)
        else:
            new_sentence.append(word)
    
    return ' '.join(new_sentence)
    return

In [38]:
sentence =  "This is a test sentence to see which words get sampled, and what is returned by the replacement function."
num_words_to_replace = 3
new_sentence = replace_words_WordNet(sentence, num_words_to_replace)
print(new_sentence)

['is', 'function', 'This', 'test', 'get', 'is']
[13, 18, 0, 3, 9, 1]
This be a prove sentence to see which words bring sampled , and what is returned by the replacement function .


In [50]:
def replace_words_ChatGPT(sentence,num_words_to_replace):
    #function to perform synonym replacement for individual words, without context provided
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Filter out non-alphabetic tokens (like punctuation)
    words_alpha = [word for word in words if word.isalpha()]

    # Randomly sample words to replace - i use 2x words just to account for words without synonym
    replacement_indices, words_to_replace = sample_words(words_alpha, num_words_to_replace)
    print(replacement_indices)
    print(words_to_replace)
    # Create a new sentence with synonyms replaced
    words_replaced = 0
    new_sentence = []
    for word in words:
        if word in words_to_replace and words_alpha.index(word) in replacement_indices and words_replaced < num_words_to_replace:
            #ask chatGPT for synonym
            new_word = ind_syn_ChatGPT(word) 
            new_sentence.append(new_word)
            #operational
            all_words.append(word)
            words_replaced +=1
        else:
            new_sentence.append(word)
    
    return ' '.join(new_sentence)
    return ' '.join(new_sentence)

In [40]:
sentence = "This is a test sentence to see which words get sampled, and what is returned by the replacement function."
num_words_to_replace = 5
replace_words_ChatGPT(sentence, 5)

[2, 13, 0, 12, 1, 8, 11, 10, 6, 16]
['a', 'is', 'This', 'what', 'is', 'words', 'and', 'sampled', 'see', 'the']


'it equals one test sentence to observe which terms get sampled , and what is returned by the replacement function .'

In [51]:
def insert_brackets(phrase, indices):
    for i in indices:
        if 0 <= i < len(phrase):
            phrase[i] = f"[{phrase[i]}]"
    new_phrase = ' '.join(phrase)
    return new_phrase

In [52]:
def replace_words_ChatGPT_context(sentence, num_words_to_replace):
#get indices of words randomly sampled from sentence/phrase
#indices are passed into llm prompting function to replace words given some context. 
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Filter out non-alphabetic tokens (like punctuation)
    words_alpha = [word for word in words if word.isalpha()]
    
    # Randomly sample words to replace - i use 2x words just to account for words without synonym
    replacement_indices, words_to_replace = sample_words(words_alpha, num_words_to_replace)
    print(replacement_indices)
    print(words_to_replace)

    phrase_to_replace = insert_brackets(words_alpha, replacement_indices)
    new_phrase = syn_from_contxt_ChatGPT(phrase_to_replace)
    return new_phrase

In [42]:
sentence = "This is a test sentence to see which words get sampled, and what is returned by the replacement function."
num_words_to_replace = 5
replace_words_ChatGPT_context(sentence, 5)
#does not always preserve punctuation at end of sentence.
#doing pairwise measurements
#make sure to save results of replacement functions in json file
#how to choose length of phrases/context window passed to the replacement functions? 


[10, 8, 15, 1, 12, 17, 5, 6, 14, 0]
['sampled', 'words', 'by', 'is', 'what', 'replacement', 'to', 'see', 'returned', 'This']


'This serves as a test sentence to determine which terms get selected and what is provided through the substitution function.'